<a href="https://colab.research.google.com/github/Amaro-Alvial-Student/GRD_Prediction/blob/main/PrediccionDelGRD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>